In [1]:
import sys
import os

sys.path.append('../src-py/')

In [16]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import json
import re

tqdm.pandas()

pd.set_option('display.max_colwidth', None)

In [3]:
from datasets import load_metric
metric = load_metric("rouge")

In [4]:
from scipy import sparse
from fast_pagerank import pagerank
from fast_pagerank import pagerank_power
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize

#from sentence_transformers import SentenceTransformer, InputExample, LoggingHandler, losses, models, util
#from transformers import BertForSequenceClassification, AutoTokenizer, TextClassificationPipeline
#import torch


In [5]:
#from claim_scoring import *

In [6]:
import tensorflow as tf
import tensorflow_hub as hub

### This notebook is to train and evalaute ArgLexRank algorithm:

- Paper: https://webis.de/downloads/publications/papers/alshomary_2020b.pdf

#### We will test different versions of argumentation scorers:
- Claim identification (https://aclanthology.org/N19-1054.pdf) (DONE)
- Claim Lexicon (https://github.com/hjshah142/SnippetGeneration/blob/master/data/ClaimLexicon.txt) Count number of words that appear in the lexicon and consider it the score of the sentence.
- Argumentation quality assessment Gertz et al. (https://arxiv.org/pdf/1911.11408.pdf). Code at (/mnt/ceph/storage/data-tmp/2021/jrieskamp/jl-workspace/bert-finetuning)

#### For each version, we will evaluate the top selected two sentences against the ground-truth of the following dataset:
https://github.com/webis-de/sigir20-extractive-snippet-generation-for-arguments/blob/master/data/dataset_as_json_file.json

In [7]:
json_data = json.load(open('../data/dataset_as_json_file.json'))

In [8]:
df=pd.DataFrame(json_data)

In [9]:
#Loading transformer-based model for the embedding
#sbert_model = SentenceTransformer('bert-base-uncased')
use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [10]:
#Loading the claim-identification pre-trained model
#claim_miner_model_path = '/mnt/ceph/storage/data-in-progress/data-research/arguana/arg-generation/claim-mining/export.pkl'
#claim_miner  = ClaimMiner(claim_miner_model_path, 'Prediction')

In [11]:
lexicon_file = open('../data/ClaimLexicon.txt')
lexicon_contents = lexicon_file.read()
lexicon_tokens = lexicon_contents.split(', ')

discourse_markers = ["for example", "such as", "for instance", "in the case of", "as revealed by",
"illustrated by",
"because", "so", "therefore", "thus", "consequently", "hence", "similariy",
"likewise",
"as with",
"like", "equally", "in the same way", "first", "second ",
"third,", "finally", "next", "meanwhile", "after", "then", "subsequently",
"above all",
"in particular", "especially", "significantly", "indeed", "notably", "but", "however",
"although",
"unless", "except", "apart from", "as long as", "if", "whereas", "instead of",
"alternatively", "otherwise", "unlike", "on the other hand", "conversely"]

lexicon_tokens +=discourse_markers

In [12]:
#Gretz et al 2019 model
# set the model with trained data
#gretz_model = BertForSequenceClassification.from_pretrained('/mnt/ceph/storage/data-tmp/2021/jrieskamp/jl-workspace/bert-finetuning/argQ-trainer/checkpoint-9000', local_files_only=True, cache_dir='cache')
# set the tokenizer
#tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', cache_dir='cache')

In [13]:
#Applying ArgLexRank to generate the gist of an argument

#1. Compute embedding for each sentence in the post
#2. Compute argumentativeness scores for each sentence in the post
#3. For each post, run PageRank algorithm on the graph constructed from sentences of the post (or from the post and all its comments)
#4. Take top two sentences to be the conclusion

REMAP = {"-LRB-": "(", "-RRB-": ")", "-LCB-": "{", "-RCB-": "}",
         "-LSB-": "[", "-RSB-": "]", "``": '"', "''": '"'}

def clean(x):
    return re.sub(
        r"-LRB-|-RRB-|-LCB-|-RCB-|-LSB-|-RSB-|``|''",
        lambda m: REMAP.get(m.group()), x)


def get_sentences_lexicon_scores(sentences):
    score_sentences = []
    for sentence in sentences:
        score = 0  # number of times a unique word from a sentence also in lexicon
        words_counted = []  # list of words that are already counted in the sentence.
        words = word_tokenize(sentence.lower())
        for word in words:
            if lexicon_tokens.__contains__(word) and not words_counted.__contains__(word):
                words_counted.append(word)
                score += 1
        score_sentences.append(score/len(words))
        
    return score_sentences

def get_argumentative_quality_scores(sentences):
    
    # set the cuda device
    if torch.cuda.device_count() >= 1:
        DEVICE = torch.cuda.current_device()
    else:
        DEVICE = -1
    
    # set the pipeline
    pipeline = TextClassificationPipeline(model=gretz_model, tokenizer=tokenizer, framework='pt', task='ArgQ')
    results = pipeline(sentences, truncation=True)
    sentences_scores = []
    for result_dict in results:
        sentences_scores.append(result_dict['score'])
    return sentences_scores

def get_arg_scores(sentences):
    claim_scores = claim_miner.predict(sentences)
    return [x[1].item() for x in claim_scores]

def gen_match_matrix(sim_method, sents, min_match_score=0):
    
    sents_embeddings  = sim_method(sents)    
    
    sim_matrix = cosine_similarity(sents_embeddings, sents_embeddings)
    super_threshold_indices = sim_matrix < min_match_score
    sim_matrix[super_threshold_indices] = 0

    return sim_matrix

def arg_rank_extract_gist(similarity_model, post, context = [], min_sim_score=0, 
                 min_arg_quality_score=0, p=0.8, argumentative_scorer='claim-identifier'):
    
    context = [s for s in context if s not in post] #filter out the argument from the context
    all_sentences = post + context
    
    #print(all_sentences)
    #limit all sentences to be of length maximium 512 tokens
#     all_esntences = [nltk.word_tokenize(s)[:512] for s in sentences]
#     all_sentences = [''.]
    
    # print('arg scorer {}'.format(argumentative_scorer))
    # print('min_arg_quality {}'.format(min_arg_quality_score))
    # print('min_sim_score {}'.format(min_sim_score))
    # print('p {}'.format(p))
    # print('context is provided {}'.format(len(context) > 0))
    
    #compute argumentativeness scores and filter out sentences with quality less htan arg_quality_score
    if argumentative_scorer   == 'claim-identifier':
        all_sentences_arg_scores = get_arg_scores(all_sentences)
    elif argumentative_scorer == 'arg-quality':
        all_sentences_arg_scores = get_argumentative_quality_scores(all_sentences)
    else:
        # represented by "lexicon_score"
        all_sentences_arg_scores = get_sentences_lexicon_scores(all_sentences)
    
    
    filtered_sentences = [x for x in zip(all_sentences, all_sentences_arg_scores) 
                          if x[1] >= min_arg_quality_score]
    

    filtered_sentences_texts, filtered_sentences_arg_scores = zip(*filtered_sentences)

    sim_matrix = gen_match_matrix(similarity_model, filtered_sentences_texts, min_match_score=min_sim_score)
    pr=pagerank(sim_matrix, personalize=np.array(filtered_sentences_arg_scores), p=p)

    ranked_candidates = list(zip(filtered_sentences_texts, pr))
    ranked_candidates = sorted(ranked_candidates, key=lambda x: -x[1])
    #keep only sentences from the post
    ranked_candidates = [x for x in ranked_candidates if x[0] in post]
    return [x[0] for x in ranked_candidates][:2]

In [14]:
def evaluate_snippets(gt_snippets, gen_snippets):
    gt_concat_snippets = [' '.join(x) for x in gt_snippets]
    gen_concat_snippets = [' '.join(x) for x in gen_snippets]

    results = metric.compute(predictions=gen_concat_snippets, references=gt_concat_snippets)
    #['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
    scores = [results["rouge1"].mid.fmeasure, results["rouge2"].mid.fmeasure, results["rougeL"].mid.fmeasure]
    
    #compute accuracy
    acc = np.mean([len(set(x[0]).intersection(set(x[1])))/2 for x in zip(gen_snippets, gt_snippets)])
    return scores + [acc]
        
def find_best_arg_technique(df, sim_method, provide_context=False, p=0.15):    
    scores = []
    for arg_scorer in ['claim-lexicon']: # ['claim-identifier', 'arg-quality', 'claim-lexicon']:
        
        generated_snippets=[]
        for idx, row in df.iterrows():
            argument = row['sentences']
            context  = [s for a in row['context'] for s in a] if provide_context else [] #flatten
            generated_snippets.append(arg_rank_extract_gist(sim_method, argument, context=context,
                                           min_arg_quality_score=0,
                                           min_sim_score=0,
                                           p=p, argumentative_scorer = arg_scorer))
                        
        rouge_scores = evaluate_snippets(df.snippet.tolist(), generated_snippets)
        scores.append([arg_scorer] + rouge_scores)
        print(scores)

    scores_df = pd.DataFrame(scores, columns = ['arg_scorer', 'R-1', 'R-2', 'R-L', 'ACC'])
    scores_df.to_csv('../data/arglex-rank-new-{}-scores.csv'.format(arg_scorer))
    return scores_df

### Fineding best argumentation technique:

- Fine the best argumentation method

In [17]:
df['sentences'] = df.sentences.apply(lambda x: [clean(sent).replace("\n", "") for sent in x])

In [18]:
#load the arg_ids from the test set
test_df = pd.read_csv('../data/arglex-rank/test_argumnets.csv')
test_arg_ids = test_df['Arg ID'].tolist()

In [19]:
arg_context_df = df.groupby('query').agg({
    'sentences': lambda x: list(x)
}).reset_index()
arg_context_dict = pd.Series(arg_context_df.sentences.values, index=arg_context_df['query'])
df['context'] = df['query'].apply(lambda x: arg_context_dict[x])

In [20]:
test_df = df[df.arg_id.isin(test_arg_ids)]

In [21]:
use_results = find_best_arg_technique(test_df, use_model, provide_context=True, p=0.15)

/usr/local/lib/python3.8/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
/usr/local/lib/python3.8/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
/usr/local/lib/python3.8/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
/usr/local/lib/python3.8/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
/usr/local/lib/python3.8/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix form

[['claim-lexicon', 0.4169489467161268, 0.31685441572586015, 0.3462914825414187, 0.32]]


In [22]:
use_results

,arg_scorer,R-1,R-2,R-L,ACC
0,claim-lexicon,0.416949,0.316854,0.346291,0.32


In [1]:
#sbert_results = find_best_arg_technique(test_df, lambda x: sbert_model.encode(x), provide_context=True, p=0.15)

In [18]:
valid_df = pd.read_pickle('/mnt/ceph/storage/data-in-progress/data-research/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/valid_conclusion_comp_remove_75sem_perc_with_targets.pkl')
unique_valid_posts = valid_df.drop_duplicates('post_id')
unique_valid_posts.head()

,post_id,split,comment_id,title,post,n_sentences,counter,num_cand_conc,masked_premises,premises_with_conclusion,conclusion_stance,conclusion_targets,bart_conclusion,arglex_rank_conclusion
295914,t3_7uek1b,val,t1_dtjoww6,Pink is just light red and doesn't deserve it's own name.,"[i understand the need for more precise naming when you're doing something artsy., what i'm focusing on is how we describe colors in everyday life., pink is just a light shade of the color red., we already have a ton of precedence for just calling something a light or dark color., when people see a light or dark blue object, they just call it light blue or dark blue., when the see a light or dark green thing, they call it light green or dark green., for some reason, when we see something that is light red we call it pink., i don't see any rhyme or reason to this distinction and i don't thi...",8,"[what's your opinion on scarlet?, brick?, salmon?, rose?, wine?]",1,"[i understand the need for more precise naming when you're doing something artsy., what i'm focusing on is how we describe colors in everyday life., pink is just a light shade of the color red., we already have a ton of precedence for just calling something a light or dark color., when people see a light or dark blue object, they just call it light blue or dark blue., when the see a light or dark green thing, they call it light green or dark green., for some reason, when we see something that is light red we call it pink., i don't see any rhyme or reason to this distinction and i don't thi...","[i understand the need for more precise naming when you're doing something artsy., what i'm focusing on is how we describe colors in everyday life., pink is just a light shade of the color red., we already have a ton of precedence for just calling something a light or dark color., when people see a light or dark blue object, they just call it light blue or dark blue., when the see a light or dark green thing, they call it light green or dark green., for some reason, when we see something that is light red we call it pink., i don't see any rhyme or reason to this distinction and i don't thi...",-0.846022,Pink,i don't think pink deserves it's own name.,i don't see any rhyme or reason to this distinction and i don't think pink deserves it's name.
295939,t3_7uj99e,val,t1_dtknvk0,Elon Musk should not be selling flamethrowers as if they were toys.,"[i just saw on the news that elon musk have sold over 20 thousand flamethrowers., they appear to be oversized blowtorches designed to be just below the limits that would require licenses in most states., in a promotional video, he jokes around with it, pretending to use it on the cameraman. on twitter, he jokes about it been useful during a zombie apocalypse., but, as far as i know, no real use for it is seriously discussed., i find this behaviour borderline negligent., if these things become even more popular, it is just a matter of time until someone gets seriously hurt., either on purpo...",14,"[people get hurt with tide pods and knives, so why would we care about this?, we can't protect every dumb person in this planet from themselves.]",1,"[i just saw on the news that elon musk have sold over 20 thousand flamethrowers., they appear to be oversized blowtorches designed to be just below the limits that would require licenses in most states., in a promotional video, he jokes around with it, pretending to use it on the cameraman. on twitter, he jokes about it been useful during a zombie apocalypse., but, as far as i know, no real use for it is seriously discussed., i find this behaviour borderline negligent., if these things become even more popular, it is just a matter of time until someone gets seriously hurt., either on purpo...","[i just saw on the news that elon musk have sold over 20 thousand flamethrowers., they appear to be oversized blowtorches designed to be just below the limits that would require licenses in most states., in a promotional video, he jok

In [29]:
unique_valid_posts['arglex_rank_conclusion'] = unique_valid_posts['masked_premises'].apply(lambda x : extract_gist(model, x, 
                                                   min_arg_quality_score=0.0,
                                                   min_sim_score=0.8,
                                                   p=0.2, argumentative_scorer = 'arg-quality'))

/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseE

In [30]:
unique_valid_posts.to_pickle('/mnt/ceph/storage/data-in-progress/data-research/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/valid_conclusion_comp_remove_75sem_perc_unique_posts_with_arglexrank_conclusions.pkl')

In [ ]:
test_df = pd.read_pickle('/mnt/ceph/storage/data-in-progress/data-research/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/test_conclusion_comp_remove_75sem_perc_with_targets.pkl')
unique_test_posts = test_df.drop_duplicates('post_id')

In [ ]:
unique_test_posts['arglex_rank_conclusion'] = unique_test_posts['masked_premises'].apply(lambda x : extract_gist(model, x, 
                                                   min_arg_quality_score=0.0,
                                                   min_sim_score=0.8,
                                                   p=0.2, argumentative_scorer = 'arg-quality'))

unique_test_posts.to_pickle('/mnt/ceph/storage/data-in-progress/data-research/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/test_conclusion_comp_remove_75sem_perc_with_targets_with_arglexrank_conclusions.pkl')

#### Generate arglex_rank conclusions for the dev_sample all:

In [ ]:
test_df = pd.read_pickle('/mnt/ceph/storage/data-in-progress/data-research/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/valid_conclusion_all_sample_with_targets.pkl')
unique_test_posts = test_df.drop_duplicates('post_id')

unique_test_posts['arglex_rank_conclusion'] = unique_test_posts['post'].apply(lambda x : extract_gist(model, x, 
                                                   min_arg_quality_score=0.0,
                                                   min_sim_score=0.8,
                                                   p=0.2, argumentative_scorer = 'arg-quality'))

unique_test_posts.to_pickle('/mnt/ceph/storage/data-in-progress/data-research/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/valid_conclusion_all_sample_with_targets_with_arglexrank_conclusions.pkl')


/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseE

In [ ]:
test_df = pd.read_pickle('/mnt/ceph/storage/data-in-progress/data-research/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/test_conclusion_all_sample_with_targets.pkl')
unique_test_posts = test_df.drop_duplicates('post_id')

unique_test_posts['arglex_rank_conclusion'] = unique_test_posts['post'].apply(lambda x : extract_gist(model, x, 
                                                   min_arg_quality_score=0.0,
                                                   min_sim_score=0.8,
                                                   p=0.2, argumentative_scorer = 'arg-quality'))

unique_test_posts.to_pickle('/mnt/ceph/storage/data-in-progress/data-research/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/test_conclusion_all_sample_with_targets_with_arglexrank_conclusions.pkl')

/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseE